# Radial Basis Network (RBN)

Радиальные базисные сети - это необычный подтип нейронной сети, который в качестве функции активации использует функцию радиального базиса (вместо логистической функции или ReLU или других линейных или нелинейных аналогов).

Радиальная базисная функция — это любая функция, которая определяется как функция расстояния от некоторой центральной точки (радиуса). Это свойство ϕ(x)=ϕ(||x||) . Есть много сигнатур функций, которые удовлетворяют этому свойству, но наиболее распространенной является гауссовская: ϕ(r)=e−(ϵr)2

Сети с радиальным базисом представляют собой полностью связанные нейронные сети с прямой связью. RBN можно использовать для функциональной аппроксимации.

Радиальные базисные сети не имеют собственного определения keras.layer. Поэтому, если нужно использовать RBN, придется самостоятельно определить необходимый слой как пользовательский слой.

Input Data
https://www.kaggle.com/code/residentmario/radial-basis-networks-and-custom-keras-layers/input

In [1]:
import pandas as pd
import numpy as np

from keras.layers import Layer
from keras import backend as K
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.losses import binary_crossentropy

In [4]:
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = K.cast_to_floatx(gamma)

    def build(self, input_shape):
        """
        This is where you will define your weights.
        This method should end with calling super([Layer], self).build()
        """
        print(input_shape)
        print(self.units)
        self.mu = self.add_weight(name='mu',
                                  shape=(int(input_shape[1]), self.units),
                                  initializer='uniform',
                                  trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        diff = K.expand_dims(inputs) - self.mu
        l2 = K.sum(K.pow(diff, 2), axis=1)
        res = K.exp(-1 * self.gamma * l2)
        return res

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.units)

# Custom non-trainable keras layers

In [6]:
X = np.load('/content/k49-train-imgs.npz')['arr_0']
y = np.load('/content/k49-train-labels.npz')['arr_0']
y = (y <= 25).astype(int)

model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(RBFLayer(10, 0.5))
model.add(Dense(1, activation='sigmoid', name='foo'))

model.compile(optimizer='rmsprop', loss=binary_crossentropy)

(None, 784)
10


In [7]:
model.fit(X, y, batch_size=256, epochs=5)

Epoch 1/5
908/908 [==============================] - 21s 22ms/step - loss: 0.6823
Epoch 2/5
908/908 [==============================] - 19s 20ms/step - loss: 0.6806
Epoch 3/5
908/908 [==============================] - 20s 22ms/step - loss: 0.6806
Epoch 4/5
908/908 [==============================] - 19s 21ms/step - loss: 0.6805
Epoch 5/5
908/908 [==============================] - 21s 24ms/step - loss: 0.6805
